In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
import platform
if platform.system() == 'Darwin':
	sys.path.append('/Users/jiadongdan/Dropbox/stempy')
else:
	sys.path.append('D:\\Dropbox\\stempy')
    
from stempy.io import *
from stempy.denoise import *
from stempy.datasets import *
from stempy.utils import *
from stempy.plot import *
from stempy.feature import *
from stempy.manifold import *
from stempy.clustering import *
from stempy.spatial import * 
from stempy.graph import *

In [181]:
X = np.load('X.npy')
ps = np.load('ps.npy')

In [210]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
def get_lbs(X, n=2, n_init=20, **kwargs):
    aic_list = []
    bic_list = []
    lbs_list = []
    lb = []
    db = []
    # covariance_type and n_init have huge effects
    covariance_types = ['full', 'spherical', 'tied', 'diag']
    #reg_covars = [1e-6, 1e-5, 1e-4, 1e-3]
    reg_covars = [1e-6, 1e-5, 1e-4, 1e-3]
    for reg_covar in reg_covars:
        for type in covariance_types:
            model = GaussianMixture(n, covariance_type=type, n_init=n_init, random_state=0, reg_covar=reg_covar,
                                    **kwargs).fit(X)
            lbs_ = model.predict(X)
            lbs_list.append(lbs_)
            aic_list.append(model.aic(X))
            bic_list.append(model.bic(X))
            lb.append(model.lower_bound_)
            print(model.predict(X), reg_covar, type)
            if np.unique(lbs_).shape[0] >= 2:
                db.append(silhouette_score(X, lbs_))
            else:
                db.append(-1)
    aic_bic = np.array(aic_list) + np.array(bic_list)
    ind = np.argmin(lb)
    ind = np.argmax(db)
    print(ind)
    lbs = lbs_list[ind]
    #plt.plot(aic_list)
    #plt.plot(bic_list)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.2, 3.6))
    ax1.plot(db)
    ax2.plot(lb)
    return lbs

In [211]:
lbs = get_lbs(X, 2)

[0 0 0 ... 0 1 1] 1e-06 full
[0 0 1 ... 0 0 0] 1e-06 spherical
[0 0 0 ... 0 0 0] 1e-06 tied
[1 1 0 ... 1 0 0] 1e-06 diag
[0 0 0 ... 0 0 0] 1e-05 full
[0 0 1 ... 0 0 0] 1e-05 spherical
[1 1 0 ... 1 1 1] 1e-05 tied
[0 0 1 ... 0 1 1] 1e-05 diag
[1 1 1 ... 1 1 1] 0.0001 full
[0 0 1 ... 0 0 0] 0.0001 spherical
[1 1 0 ... 1 1 1] 0.0001 tied
[0 1 1 ... 0 1 1] 0.0001 diag
[0 0 0 ... 0 0 0] 0.001 full
[1 1 1 ... 1 1 1] 0.001 spherical
[1 1 1 ... 1 1 1] 0.001 tied
[1 1 1 ... 1 1 1] 0.001 diag
2


In [213]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(pca(X)[:, 0], pca(X)[:, 1], color=colors_from_lbs(lbs), s=10) 

In [156]:
plot_pca(X, 3, lbs=lbs)c

In [201]:
silhouette_score(pca(X), lbs)

0.2598545955382636